In [1]:
from Eval_basis import *
import pandas as pd

[2019-08-13 12:08:34,318] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [2]:
np.random.seed(1)
%matplotlib inline

In [3]:
data_full = EbiData("./data", experiment="E-ENAD-15")

[2019-08-13 12:09:37,657] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2019-08-13 12:09:37,720] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-13 12:09:37,722] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-13 12:09:38,782] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-13 12:09:39,822] INFO - scvi.dataset.dataset | Downsampled from 50896 to 35577 cells


In [4]:
data_big_mapped = UnionDataset("./data", map_fname="ensembl_mouse_genes-proteincoding", low_memory=False)
data_big_mapped.union_from_memory([data_full])
data_big_mapped.filter_cell_types(np.array([ct for ct in data_big_mapped.cell_types if ct != "not available"]))

[2019-08-13 12:10:42,485] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-13 12:10:42,487] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-13 12:10:42,491] INFO - scvi.dataset.dataset | Joined 1 datasets to one of shape 35577 x 22250.
[2019-08-13 12:10:43,476] INFO - scvi.dataset.dataset | Downsampled from 35577 to 31394 cells


In [5]:
agg = data_full.obs.groupby(["Sample Characteristic[organism part]", "cell_types"]).size()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(agg.sort_index())

Sample Characteristic[organism part]      cell_types                                        
aorta                                     endothelial cell                                       188
                                          erythrocyte                                             91
                                          fibroblast                                              70
                                          not available                                          694
                                          professional antigen presenting cell                    59
ascending colon                           Brush cell of epithelium proper of large intestine      53
                                          enterocyte of epithelium of large intestine            761
                                          enteroendocrine cell                                    29
                                          epithelial cell of large intestine                    141

In [ ]:
n_epochs = 100
colors=None
for tissue in np.unique(agg.index.get_level_values(0)): 
    print("Training VAE for tissue ", tissue)
    cutout_cts = agg[tissue].index.values
    if "not available" in cutout_cts:
        cutout_cts = np.delete(cutout_cts, np.where(cutout_cts == "not available")[0])

    rem_ct = data_big_mapped.cell_types[~np.isin(data_big_mapped.cell_types, cutout_cts)]
    data_small = copy.deepcopy(data_big_mapped)
    data_big = copy.deepcopy(data_big_mapped)

    data_small.filter_cell_types(cutout_cts)
    data_big.filter_cell_types(rem_ct)
    
    trainer_big = train_vae(data_big, "./data", f"big_{tissue}_data_portion", n_epochs=n_epochs)
    trainer_small = train_vae(data_small, "./data", f"small_{tissue}_data_portion", n_epochs=n_epochs)
    dot_size = (mpl.rcParams['lines.markersize'] ** 2.0)

    posterior_big = plot_tsne(trainer_big, trainer_big.model, data_big, f"./plots/big_{tissue}_data_portion",
                              colors=colors, s=dot_size, edgecolors='black')
    posterior_small = plot_tsne(trainer_small, trainer_small.model, data_small, f"./plots/small_{tissue}_data_portion",
                                colors=colors, s=dot_size, edgecolors='black')
    posterior_small_in_big = plot_tsne(trainer_big, trainer_big.model, data_small, f"./plots/small_{tissue}_data_portion_in_big",
                                       colors=colors, s=dot_size, edgecolors='black')

Training VAE for tissue  aorta


[2019-08-13 12:10:43,861] INFO - scvi.dataset.dataset | Downsampled from 31394 to 3985 cells
[2019-08-13 12:10:44,706] INFO - scvi.dataset.dataset | Downsampled from 31394 to 27409 cells


Training VAE for tissue  ascending colon


[2019-08-13 12:11:56,022] INFO - scvi.dataset.dataset | Downsampled from 31394 to 2725 cells
[2019-08-13 12:11:56,850] INFO - scvi.dataset.dataset | Downsampled from 31394 to 28669 cells


Training VAE for tissue  back skin


...

training: 100%|██████████| 100/100 [00:30<00:00,  2.81it/s]


/home/michael/GitHub/scVI_ma/scvi/inference/posterior.py:943: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(16, 16))
/home/michael/GitHub/scVI_ma/scvi/inference/posterior.py:943: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(16, 16))
/home/michael/GitHub/scVI_ma/scvi/inference/posterior.py:943: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control thi

Training VAE for tissue  urinary bladder


[2019-08-13 15:02:45,664] INFO - scvi.dataset.dataset | Downsampled from 31394 to 982 cells
INFO:scvi.dataset.dataset:Downsampled from 31394 to 982 cells
[2019-08-13 15:02:46,576] INFO - scvi.dataset.dataset | Downsampled from 31394 to 30412 cells
INFO:scvi.dataset.dataset:Downsampled from 31394 to 30412 cells


training:  25%|██▌       | 25/100 [01:14<04:16,  3.42s/it]

In [ ]:
data_big_mapped.X

In [ ]:
data_big.gene_names

In [ ]:
np.isin(data_full.gene_names, data_big_mapped.gene_names).sum() / len(data_full.gene_names)